In [22]:
import subprocess
import sys
subprocess.check_call([sys.executable, "-m", "pip", "install", "fastapi", "uvicorn", "httpx", "langgraph", "langchain"])

import uuid
from typing import Dict, Any, List, TypedDict, Annotated
from fastapi import FastAPI
from fastapi.testclient import TestClient
from pydantic import BaseModel
from langgraph.graph import StateGraph, END

In [23]:
def smart_summarize(text: str) -> str:
    if "." in text:
        return text.split(".")[0] + "."
    return text

def smart_refine(text: str) -> str:
    fillers = ["stateful,", "multi-actor", "expression", "application", "concept", "very", "highly"]
    words = text.split()
    for filler in fillers:
        if filler in text:
            return text.replace(filler, "").replace("  ", " ").strip()
    if len(words) > 3:
        return " ".join(words[:-1]) + "."
    return text


In [24]:
class AgentState(TypedDict):
    input_text: str
    chunks: List[str]
    chunk_summaries: List[str]
    final_summary: str
    length_limit: int
    steps: int

def split_text(state: AgentState):
    text = state.get("input_text", "")
    chunks = [s.strip() for s in text.split('.') if len(s) > 5]
    return {"chunks": chunks, "steps": 1}

def generate_summaries(state: AgentState):
    chunks = state.get("chunks", [])
    summaries = [smart_summarize(c) for c in chunks]
    return {"chunk_summaries": summaries, "steps": state["steps"] + 1}

def merge_summaries(state: AgentState):
    summaries = state.get("chunk_summaries", [])
    merged = " ".join(summaries)
    return {"final_summary": merged, "steps": state["steps"] + 1}

def refine_summary(state: AgentState):
    current = state.get("final_summary", "")
    refined = smart_refine(current)
    return {"final_summary": refined, "steps": state["steps"] + 1}

def length_gate(state: AgentState):
    summary = state.get("final_summary", "")
    limit = state.get("length_limit", 50)
    steps = state.get("steps", 0)

    print(f"   [Check] Length: {len(summary)} (Limit: {limit}) -> Content: {summary}")

    if steps > 15:
        return END

    if len(summary) > limit:
        return "refine"
    return END

def build_workflow():
    workflow = StateGraph(AgentState)

    workflow.add_node("split", split_text)
    workflow.add_node("summarize", generate_summaries)
    workflow.add_node("merge", merge_summaries)
    workflow.add_node("refine", refine_summary)

    workflow.set_entry_point("split")
    workflow.add_edge("split", "summarize")
    workflow.add_edge("summarize", "merge")
    workflow.add_edge("merge", "refine")

    workflow.add_conditional_edges(
        "refine",
        length_gate,
        {
            "refine": "refine",
            END: END
        }
    )

    return workflow.compile()


In [25]:
app = FastAPI()
client = TestClient(app)
graphs = {}

@app.post("/graph/run")
def run_graph(payload: dict):
    app_runnable = build_workflow()

    inputs = payload.get("initial_state")
    if "steps" not in inputs:
        inputs["steps"] = 0

    final_state = app_runnable.invoke(inputs)
    return {
        "final_summary": final_state.get("final_summary"),
        "steps_taken": final_state.get("steps")
    }


In [27]:
input_text = "LangGraph is a library for building stateful, multi-actor applications with LLMs. It is highly useful."

payload = {
    "initial_state": {
        "input_text": input_text,
        "length_limit": 65
    }
}

response = client.post("/graph/run", json=payload)
result = response.json()

print(f"Original Text: {input_text}")
print(f"Final Summary: {result['final_summary']}")
print(f"Steps Taken:   {result['steps_taken']}")


   [Check] Length: 90 (Limit: 65) -> Content: LangGraph is a library for building multi-actor applications with LLMs It is highly useful
   [Check] Length: 78 (Limit: 65) -> Content: LangGraph is a library for building applications with LLMs It is highly useful
   [Check] Length: 67 (Limit: 65) -> Content: LangGraph is a library for building s with LLMs It is highly useful
   [Check] Length: 60 (Limit: 65) -> Content: LangGraph is a library for building s with LLMs It is useful
Original Text: LangGraph is a library for building stateful, multi-actor applications with LLMs. It is highly useful.
Final Summary: LangGraph is a library for building s with LLMs It is useful
Steps Taken:   7
